In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2240589" #@param {type:"string"}

In [2]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "9886499911" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()

notebook= "M3_AST_24_Pretrained_models_for_Oxford_IIIT_pet_dataset_C" #name of the notebook

def setup():
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/oxfordiitimage.zip")
    ipython.magic("sx unzip oxfordiitimage.zip")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError
    else:
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError
    else:
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")

In [4]:
import numpy as np
import pandas as pd
import os,shutil,glob,PIL
import pathlib
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm

In [5]:
# Normalize with mean and std
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.4839, 0.4528, 0.3962), (0.2702, 0.2655, 0.2745))])

In [6]:
# Loading the train set file
train_data_folder = "/content/oxfordiitimage/train" # Train directory for loading images
train_data = datasets.ImageFolder(root=train_data_folder, transform=transform)

In [7]:
# Initializing batch size
batch_size = 64

# Loading the train dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [8]:
# Generate a batches of images and labels
train_images, train_labels = next(iter(train_loader))
train_images.shape, train_labels.shape

In [9]:
# labels Translator
label_names = {v: k for k, v in train_data.class_to_idx.items()}
label_names

In [10]:
# Create a grid of images along with their corresponding labels
L = 3
W = 3

fig, axes = plt.subplots(L, W, figsize = (10, 10))
axes = axes.reshape(-1)

for i in np.arange(0, L*W):
    axes[i].imshow(train_images[i].permute(1, 2, 0))
    axes[i].set_title(label_names[train_labels[i].item()])
    axes[i].axis('on')

plt.tight_layout()

In [11]:
# mean = 0.0
# for img, _ in train_data:
#   mean += img.mean([1,2])
# mean = mean/len(train_data)
# print(mean)

In [12]:
# sumel = 0.0
# countel = 0
# for img, _ in train_data:
#     img = (img - mean.unsqueeze(1).unsqueeze(1))**2
#     sumel += img.sum([1, 2])
#     countel += torch.numel(img[0])
# std = torch.sqrt(sumel/countel)
# print(std)

In [13]:
# No of Categories
len(train_data.classes)

In [14]:
# Number of training samples
len(train_data)

In [15]:
# Size of one training image
train_data[0][0].size()

In [16]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

In [17]:
# To test whether GPU instance is present in the system of not.
use_cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', use_cuda)

In [18]:
device = torch.device("cuda" if use_cuda else "cpu")
device

In [19]:
!nvidia-smi

In [20]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 37)
model_ft = model_ft.to(device)
print(model_ft)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

In [22]:
train_accu = []     # Empty list for saving train accuracy
train_losses = []   # Empty list for saving train losses

def train(epoch):

  print('\nEpoch : %d'%epoch)

  model_ft.train()    # Initiate the model in training mode

  running_loss=0
  correct=0
  total=0

  # Loop through each batch of images in train set
  for data in tqdm(train_loader):

    inputs, labels = data[0].to(device), data[1].to(device)   # Loading the input tensors into CUDA GPU

    # Zero out the gradients from the preivous step
    optimizer.zero_grad()

    # Forward pass (this calls the "forward" function within model_ft)
    outputs = model_ft(inputs)

    # Calculating the loss
    loss = criterion(outputs, labels)

    # Back Propagation for calculaing gradients and adjusting weights
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    # Picking the class/label with maximum probability
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()


  train_loss = running_loss/len(train_loader)     # Calculating the mean of training loss
  accu = 100.*correct/total                       # Calculating the accuracy

  train_accu.append(accu)
  train_losses.append(train_loss)
  print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))

In [23]:
epochs = 20
for epoch in range(1, epochs+1):
  train(epoch)

In [24]:
PATH='/content/Oxford-IIIT-ResNet50.pth'
torch.save(model_ft.state_dict(), PATH)

In [25]:
plt.plot(train_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train'])
plt.show()

In [26]:
plt.plot(train_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend(['Train'])
plt.title('Train Loss')
plt.show()

In [27]:
PATH = '/content/Oxford-IIIT-ResNet50.pth'
model_ft.load_state_dict(torch.load(PATH))

In [28]:
model_ft.eval()

In [29]:
# Load the testset
test_data_folder = "/content/oxfordiitimage/test" # Train directory for loading images
test_data = datasets.ImageFolder(root=test_data_folder, transform=transform)

In [30]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)

In [31]:
predictions = []
labels = []
j = 0

# Loop through the test set
for i, data in enumerate(test_loader):
  input, label = data
  # print(input.shape ,test_data.classes[label])

  # Forward pass (this calls the "forward" function within model_ft)
  output = model_ft(input.to(device))

  # Output as max probability
  pred = torch.max(output,1)

  # Storing the actuals and the predictions
  labels.append(test_data.classes[label])
  predictions.append(test_data.classes[pred.indices])
  if test_data.classes[pred.indices] == test_data.classes[label]:
    j+=1

In [32]:
print(j/len(test_loader))

In [33]:
results = pd.DataFrame()
results['Labels'] = pd.Series(labels)
results['Predictions'] = pd.Series(predictions)

In [34]:
# Here we are creating a column to show whether prediction is Correct/Wrong
results['Correct/Wrong'] = np.where((results['Labels'] == results['Predictions']) , "Correct", "Wrong")

In [35]:
results.to_csv("ResNet50_Predictions.csv")

In [36]:
wrong_prediction = results[results['Correct/Wrong'] == "Wrong"]
wrong_prediction.head()

In [37]:
results["Correct/Wrong"].value_counts()

In [38]:
correct_prediction = results[results['Correct/Wrong'] == "Correct"]
correct_prediction.head()

In [39]:
model_ft = models.googlenet(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,37)
model_ft = model_ft.to(device)
print(model_ft)

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

In [41]:
train_accu = []     # Empty list for saving train accuracy
train_losses = []   # Empty list for saving train losses
def train(epoch):
  print('\nEpoch : %d'%epoch)

  model_ft.train()    # Initiate the model in training mode

  running_loss = 0
  correct = 0
  total = 0

  for data in tqdm(train_loader):

    inputs,labels=data[0].to(device),data[1].to(device)   # Loading the input tensors into CUDA GPU

    optimizer.zero_grad()
    outputs = model_ft(inputs)
    loss = criterion(outputs,labels)  # Calculating the loss
    loss.backward()                   # Back Propagation for calculaing gradients and adjusting weights
    optimizer.step()

    running_loss += loss.item()

    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()


  train_loss = running_loss/len(train_loader)     # Calculating the mean of training loss
  accu = 100.*correct/total                       # Calculating the accuracy

  train_accu.append(accu)
  train_losses.append(train_loss)
  print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))

In [42]:
epochs = 20
for epoch in range(1, epochs+1):
  train(epoch)

In [43]:
PATH = '/content/Oxford-IIIT-GoogleNet.pth'
torch.save(model_ft.state_dict(), PATH)

In [44]:
plt.plot(train_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train'])
plt.show()

In [45]:
plt.plot(train_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend(['Train'])
plt.title('Train Loss')
plt.show()

In [46]:
PATH='/content/Oxford-IIIT-GoogleNet.pth'
model_ft.load_state_dict(torch.load(PATH))

In [47]:
model_ft.eval()

In [48]:
predictions = []
labels = []
j = 0
for i,data in enumerate(test_loader):
  input,label = data
  output = model_ft(input.to(device))
  pred = torch.max(output,1)
  labels.append(test_data.classes[label])
  predictions.append(test_data.classes[pred.indices])
  if test_data.classes[pred.indices] == test_data.classes[label]:
    j+=1

In [49]:
print(j/len(test_loader))

In [50]:
results = pd.DataFrame()
results['Labels'] = pd.Series(labels)
results['Predictions'] = pd.Series(predictions)

In [51]:
# Here we are creating a column to show whether prediction is Correct/Wrong
results['Correct/Wrong'] = np.where((results['Labels'] == results['Predictions']) , "Correct", "Wrong")

In [52]:
results.to_csv("GoogleNet_Predictions.csv")

In [53]:
wrong_prediction = results[results['Correct/Wrong'] == "Wrong"]
wrong_prediction.head()

In [54]:
results["Correct/Wrong"].value_counts()

In [55]:
correct_prediction = results[results['Correct/Wrong'] == "Correct"]
correct_prediction.head()

In [56]:
#@title Q1. Which of the above statements is/are True regrading transfer learning?
Answer1 = "A, B, C and D" #@param ["", "Only A and B", "Only C and D", "Only A, B and D", "Only B, C and D", "Only A, B and C", "A, B, C and D"]

In [57]:
#@title Q2. A problem with deep convolutional neural networks is that the number of feature maps often increases with the depth of the network. This problem can result in a dramatic increase in the number of parameters and computation required when larger filter sizes are used, such as 5×5 and 7×7. To address this problem, a 1×1 convolutional layer can be used that offers a channel-wise pooling, often called feature map pooling. This simple technique can be used for dimensionality reduction, decreasing the number of feature maps while retaining their salient features.
Answer2 = "FALSE" #@param ["","TRUE","FALSE"]

In [58]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]

In [59]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "na" #@param {type:"string"}

In [60]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]

In [61]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]

In [62]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]